### 최종

In [1]:
## datadam_sales_per_income

## 확인사항
## 1. KT데이터 업데이트 여부
## 2. 건축물대장 업데이트 여부
## 3. KCB 통근데이터 업데이트 여부

# module
import psycopg2
import pandas as pd

# 기준월
date = '202210'
# dates = pd.date_range('2022-02',periods=8,freq='M').strftime("%Y%m")

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# INSERT
aws_cursor.execute(
"""
insert into m2.datadam_sales_per_income (
with date as (
    select '""" + date + """'::text as base_ym
)
select (select base_ym from date) as base_ym
     , T1.pnu
     , substring(T1.pnu, 1, 8) as emd_cd
     , T1.industry_cd
     , round(1.0 * T1.sales / T4.avg_mon_income, 2) as sales
     , sysdate::date as create_at
     , sysdate::date as update_at
     , 'du.Park' as work_user
from (
    select substring(bd_mgt_sn, 1, 19) as pnu
         , ftc_cate2_cd as industry_cd
         , avg(pred_slng_amt) as sales
    from m1.kt_bldg_sales 
    where 1=1
        and base_ym = (select base_ym from date) -- 기준월
    group by 1, 2
) T1
join m2.cremao_land T2
    on T1.pnu = T2.pnu
join ( -- (9 월까지 있음)
    select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
    from m1.bld_rgst
    where 1=1
        and base_ym = (select base_ym from date) -- 기준월
) T3
    on T1.pnu = T3.pnu
left join ( -- (9 월까지 있음)
    select bs_yr_mon as base_ym
         , cur_lgl_dong_cd as emd_cd
         , round(sum(avg_mon_income * case when pop_cnt = -1 then 1.5 else pop_cnt end) / sum(case when pop_cnt = -1 then 1.5 else pop_cnt end) * 1000, 0) as avg_mon_income
    from m1.kcb_stat_cm
    where bs_yr_mon = (select base_ym from date)
    group by 1, 2
) T4
    on substring(T1.pnu, 1, 8) = T4.emd_cd
);
"""
)
aws_conn.commit()
print("Done")

Done


###### ver1

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'

# 기준월
base_ym = '202209'

### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 배후지 인구 소득대비 상업용 부동산 업종별 추정 매출액
sql = f'''
with date as (
	select {base_ym}::text as base_ym
)
select T1.base_ym
	 , T1.pnu
	 , substring(T1.pnu, 1, 8) as emd_cd
	 , ftc_cate2_cd
	 , round(1.0 * pred_slng_amt / (income * 1000), 2) as sales_per_income
from (
	select A.base_ym
		 , A.pnu
		 , sum(nvl(E.pnu_pop_cnt, 0) * nvl(E.emd_mon_income, 0) + nvl(F.pnu_pop_cnt, 0) * nvl(F.emd_mon_income, 0)) / sum(nvl(E.pnu_pop_cnt, 0) + nvl(F.pnu_pop_cnt, 0)) as income
	from (
		select distinct base_ym, substring(bd_mgt_sn, 1, 19) as pnu 
		from m1.kt_bldg_sales
		where 1=1
			and base_ym = (select base_ym from date) -- 기준월 조건
	) A -- KT PNU 유니크하게 추출
	join m2.cremao_land B -- 유효 PNU 가져오기
		on A.pnu = B.pnu
	join (
		select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
		from m1.bld_rgst
		where base_ym = (select base_ym from date)
	) C -- 건물이 있는 PNU 가져오기
		on A.pnu = C.pnu
	left join (
		select pnu, poly
		from m2.cremao_land
	) D -- 모집단 PNU 기준 반경 100m 내 PNU 모두 가져오기
		on ST_DistanceSphere(ST_Centroid(B.poly::geometry), ST_Centroid(D.poly::geometry)) < 100
	left join m2.live_pop E -- 반경 100m 거주인구 -> D의 PNU랑 조인해서 가져오기
		on D.pnu = E.pnu
	left join m2.work_pop F -- 반경 100m 직장인구 -> D의 PNU랑 조인해서 가져오기
		on D.pnu = F.pnu
	group by 1, 2
) T1 -- PNU 반경 100m 소득 계산
left join (
	select *
	from m1.kt_bldg_sales
	where base_ym = (select base_ym from date)
) T2 -- PNU 업종별 매출액
	on T1.pnu = substring(T2.bd_mgt_sn, 1, 19) 
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['base_ym','pnu','emd_cd','industry_cd','sales_per_income']

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'

In [ ]:
df

In [ ]:
df = df.drop(columns=['base_ym'])

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_income'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

In [ ]:
df